In [2]:
import os

In [4]:
# Путь к папке с аннотациями
annotation_file = r"C:\Users\abram\DataspellProjects\FinetuningEasyOCR\dataset\ch4_test_localization_transcription_gt"

# Путь для сохранения результата
output_file = "extracted_words_all_test.txt"

# Словарь для хранения слов по изображениям
image_to_words = {}

# Обход всех файлов в папке
for file_name in os.listdir(annotation_file):
    if file_name.endswith(".txt"):  # Проверяем, что это текстовый файл
        file_path = os.path.join(annotation_file, file_name)
        image_name = file_name.replace("gt_", "").replace(".txt", ".jpg")  # Преобразуем имя файла в имя изображения
        with open(file_path, "r", encoding="utf-8") as file:
            lines = file.readlines()
            words = []
            for line in lines:
                # Разделяем строку на координаты и слово
                parts = line.strip().split(",")
                if len(parts) >= 9:  # Проверяем, есть ли слово (9-я часть)
                    word = parts[-1]  # Последняя часть строки — это слово
                    if word != "###":  # Исключаем строки с '###'
                        words.append(word)
            image_to_words[image_name] = words

# Сохраняем все извлечённые слова в файл с привязкой к изображениям
with open(output_file, "w", encoding="utf-8") as out_file:
    for image_name, words in image_to_words.items():
        out_file.write(f"{image_name}: {' '.join(words)}\n")

print(f"Слова из всех файлов успешно извлечены и сохранены в файл: {output_file}")

Слова из всех файлов успешно извлечены и сохранены в файл: extracted_words_all_test.txt


In [6]:
def convert_to_yolo_format(x_min, y_min, x_max, y_max, img_width, img_height):
    # Вычисляем центр bounding box
    x_center = (x_min + x_max) / 2.0
    y_center = (y_min + y_max) / 2.0

    # Вычисляем ширину и высоту bounding box
    width = x_max - x_min
    height = y_max - y_min

    # Нормализуем координаты относительно размеров изображения
    x_center /= img_width
    y_center /= img_height
    width /= img_width
    height /= img_height

    return x_center, y_center, width, height

def process_annotations(annotations, img_width, img_height):
    yolo_annotations = []
    for ann in annotations:
        x_min, y_min, x_max, y_max, label = ann
        x_center, y_center, width, height = convert_to_yolo_format(x_min, y_min, x_max, y_max, img_width, img_height)
        yolo_annotations.append((label, x_center, y_center, width, height))
    return yolo_annotations

def process_folder(annotations_folder, images_folder, output_folder, img_width, img_height):
    # Создаем папку для выходных файлов, если она не существует
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Проходим по всем файлам аннотаций в папке
    for ann_file in os.listdir(annotations_folder):
        if ann_file.endswith(".txt"):
            # Полный путь к файлу аннотаций
            ann_path = os.path.join(annotations_folder, ann_file)

            # Читаем аннотации из файла
            with open(ann_path, 'r') as f:
                annotations = []
                for line in f:
                    parts = line.strip().split()
                    # Удаляем запятые и преобразуем в числа
                    x_min = int(parts[0].replace(',', ''))
                    y_min = int(parts[1].replace(',', ''))
                    x_max = int(parts[2].replace(',', ''))
                    y_max = int(parts[3].replace(',', ''))
                    label = parts[4].strip('"')
                    annotations.append((x_min, y_min, x_max, y_max, label))

            # Преобразуем аннотации в формат YOLO
            yolo_annotations = process_annotations(annotations, img_width, img_height)

            # Сохраняем результат в новый файл
            output_path = os.path.join(output_folder, ann_file)
            with open(output_path, 'w') as f:
                for ann in yolo_annotations:
                    label, x_center, y_center, width, height = ann
                    f.write(f"{label} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")


# Пример использования
annotations_folder = r"C:\Users\abram\DataspellProjects\FinetuningEasyOCR\archive\val\true"  # Папка с исходными аннотациями
images_folder = r"C:\Users\abram\DataspellProjects\FinetuningEasyOCR\archive\val\images"            # Папка с изображениями (для получения размеров)
output_folder = r"C:\Users\abram\DataspellProjects\FinetuningEasyOCR\archive\val\labels"  # Папка для сохранения аннотаций в формате YOLO

# Предположим, что размеры изображения 640x480
img_width = 640
img_height = 480

# Обрабатываем все файлы в папке
process_folder(annotations_folder, images_folder, output_folder, img_width, img_height)

print(f"Аннотации преобразованы и сохранены в папку: {output_folder}")

Аннотации преобразованы и сохранены в папку: C:\Users\abram\DataspellProjects\FinetuningEasyOCR\archive\val\labels
